<a href="https://colab.research.google.com/github/ykropivn/rpm2023_text_mining_workshop/blob/main/rpm_2023_text_mining_workshop_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Mining with Python

In [ ]:
# Test
2 + 2

## Data Load

In [ ]:
import wget

In [ ]:
# Install missing packages
!pip install wget

In [ ]:
import wget
import zipfile
import pandas as pd

In [ ]:
# Define the remote file to retrieve
remote_url = (
    'https://static.nhtsa.gov/nhtsa/downloads/CISS/2018/CISS_2018_CSV_files.zip'
)

# Define the local filename to save 2018 data
local_file = 'CISS_2018_CSV_files.zip'

# Make request for remote file data
wget.download(remote_url, local_file)

# Read CRASH.CSV file into pandas dataframe
df_zip_18 = zipfile.ZipFile('CISS_2018_CSV_files.zip')
raw_data_18 = pd.read_csv(df_zip_18.open('CRASH.CSV'), encoding='unicode_escape')

# 2019
remote_url = (
    'https://static.nhtsa.gov/nhtsa/downloads/CISS/2019/CISS_2019_CSV_files.zip'
)
local_file = 'CISS_2019_CSV_files.zip'
wget.download(remote_url, local_file)
df_zip_19 = zipfile.ZipFile('CISS_2019_CSV_files.zip')
raw_data_19 = pd.read_csv(df_zip_19.open('CRASH.CSV'), encoding='unicode_escape')

# 2020
remote_url = (
    'https://static.nhtsa.gov/nhtsa/downloads/CISS/2020/CISS_2020_CSV_files.zip'
)
local_file = 'CISS_2020_CSV_files.zip'
wget.download(remote_url, local_file)
df_zip_20 = zipfile.ZipFile('CISS_2020_CSV_files.zip')
raw_data_20 = pd.read_csv(df_zip_20.open('CRASH.csv'), encoding='unicode_escape')

# 2021
remote_url = (
    'https://static.nhtsa.gov/nhtsa/downloads/CISS/2021/CISS_2021_CSV_files.zip'
)
local_file = 'CISS_2021_CSV_files.zip'
wget.download(remote_url, local_file)
df_zip_21 = zipfile.ZipFile('CISS_2021_CSV_files.zip')
raw_data_21 = pd.read_csv(df_zip_21.open('CRASH.csv'), encoding='unicode_escape')

# Concatinate all years and reindex 2018-2021
raw_data = pd.concat([raw_data_18, raw_data_19, raw_data_20, raw_data_21])
raw_data.reset_index(inplace=True, drop=True)

raw_data.head(3)

In [ ]:
# Let's check columns available
raw_data.info()

In [ ]:
# Select subset of columns for analysis
case_text = raw_data[
    [
        'CASENUMBER',
        'CRASHYEAR',
        'CRASHMONTH',
        'CRASHTIME',
        'DAYOFWEEK',
        'CATEGORY',
        'CINJURED',
        'CINJSEV',
        'SUMMARY',
    ]
]

# Check top 3 rows
case_text.head(3)

In [ ]:
# Check bottom 3 rows
case_text.tail(3)

In [ ]:
# There is one crash report in the dataset with a null case number
case_text['CASENUMBER'][2169]

In [ ]:
# Let's drop it and re-index the dataframe 
case_text = case_text.dropna()
case_text.reset_index(inplace=True, drop=True) 

## Text Preprocessing

In [ ]:
# Turning all warnings off
import warnings
warnings.filterwarnings("ignore")

pd.options.mode.chained_assignment = None

In [ ]:
# Text preprocessing - Lower case transformation
case_text['text_low'] = case_text['SUMMARY'].astype(str).str.lower()
case_text.head(5)

In [ ]:
# Text preprocessing - tockenize
from nltk.tokenize import RegexpTokenizer

regexp = RegexpTokenizer('\w+')

case_text['text_token']=case_text['text_low'].apply(regexp.tokenize)
case_text.head(3)

In [ ]:
# Text preprocessing - stop words removal
import nltk

nltk.download('stopwords')

# Make a list of english stopwords
stopwords = nltk.corpus.stopwords.words('english')

# Extend the list with your own custom stopwords
my_stopwords = []
stopwords.extend(my_stopwords)

case_text['text_token_stop'] = case_text['text_token'].apply(
    lambda x: [item for item in x if item not in stopwords]
)
case_text.head(3)

In [ ]:
# Text preprocessing - stemming
from nltk.stem.porter import *
# from nltk.stem.snowball import *

stemmer = PorterStemmer() # another option is SnowballStemmer(language='english'), which performs stemming for different languages

case_text['text_token_stop_stem'] = case_text['text_token_stop'].apply(
    lambda x: [stemmer.stem(y) for y in x]
)
case_text.head(3)

In [ ]:
# Text preprocessing - filter by length (shorter than 2 symbols)
case_text['text_string'] = case_text['text_token_stop_stem'].apply(
    lambda x: ' '.join([item for item in x if len(item) > 2])
)
case_text.head(3)

In [ ]:
# Merging all words together and split them into tokens to perform frequency analysis
all_words = ' '.join([word for word in case_text['text_string']])

from nltk.tokenize import word_tokenize

nltk.download('punkt')
tokenized_words = word_tokenize(all_words)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
from nltk.probability import FreqDist

# Creating FreqDist, keeping the 10 most common tokens
fdist = FreqDist(tokenized_words).most_common(10)

# Conversion to Pandas series via Python Dictionary for easier plotting
fdist = pd.Series(dict(fdist))

# Setting figure, ax into variables
fig, ax = plt.subplots(figsize=(10, 5))

# Seaborn plotting using Pandas attributes + xtick rotation for ease of viewing
all_plot = sns.barplot(x=fdist.values, y=fdist.index, orient='h', ax=ax)
plt.title('Word Frequency Distribution')

In [ ]:
# Creating FreqDist, keeping the 20 least common tokens
least_common = FreqDist(dict(FreqDist(tokenized_words).most_common()[-20:]))

# Conversion to Pandas series via Python Dictionary for easier plotting
least_common = pd.Series(dict(least_common))

# Setting figure, ax into variables
fig, ax = plt.subplots(figsize=(10,8))

# Seaborn plotting using Pandas attributes + xtick rotation for ease of viewing
all_plot = sns.barplot(x=least_common.values, y=least_common.index, orient='h', ax=ax)
plt.title('Word Frequency Distribution')

In [ ]:
# Text preprocessing - filtering by overall word frequency
fdist = FreqDist(tokenized_words)

case_text['text_string_token']=case_text['text_string'].apply(regexp.tokenize)

case_text['text_string_fdist'] = case_text['text_string_token'].apply(
    lambda x: ' '.join(
        [item for item in x if (fdist[item] >= 1 and fdist[item] <= 14000)]
    )
)
case_text.head(3)

## Word2Vec Model

In [ ]:
from gensim.models import Word2Vec

In [ ]:
# Word2Vec - A function to build corpus
def build_corpus(data):
    corpus = []
    for doc in data.iteritems():
        word_list = doc[1].split(' ')
        corpus.append(word_list)
    return corpus

In [ ]:
corpus = build_corpus(case_text['text_string_fdist'])

In [ ]:
corpus[0]

In [ ]:
model_10 = Word2Vec(corpus, size=10, min_count=1)

In [ ]:
# After training the word2vec model, we can obtain the word embedding directly from the training model as following
model_10['deer']

In [ ]:
model_10['anim']

In [ ]:
# Let's plot 5 word-vectors to see if the representation makes sense
import plotly.graph_objects as go

fig = go.Figure(
    data=go.Heatmap(
        z=[model_10['control'], model_10['path'], model_10['anim'], model_10['deer'], model_10['run']],
        x=[
            'dim1',
            'dim2',
            'dim3',
            'dim4',
            'dim5',
            'dim6',
            'dim7',
            'dim8',
            'dim9',
            'dim10',
        ],
        y=['control', 'path', 'anim', 'deer', 'run'],
    )
)
fig.show()

In [ ]:
#  Model understanding - We can obtain the word based on similarity
sim_words = model_10.wv.most_similar('deer')
sim_words

In [ ]:
#  Model understanding - We can obtain the word based on similarity
sim_words = model_10.wv.most_similar('injur')
sim_words

In [ ]:
#  Model understanding - function to visualize the model in 2D
#  TSNE - t-distributed stochastic neighbor embedding

import numpy as np
from sklearn.manifold import TSNE
import plotly.offline as py


def tsne_plot(model):
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tokens = np.asarray(tokens)
    # Dimensionality reduction
    tsne_model = TSNE(perplexity=5, n_components=2, init='pca', n_iter=2500, random_state=123)
    new_values = tsne_model.fit_transform(tokens)

    traceTSNE = go.Scatter(
        x=new_values[:, 0],
        y=new_values[:, 1],
        mode='markers+text',
        text=labels,
        marker=dict(
            size=8,
            colorscale='Jet',
            showscale=False,
            line=dict(width=2, color='rgb(255, 255, 255)'),
            opacity=0.8,
        ),
    )
    data = [traceTSNE]

    layout = dict(
        hovermode='closest',
        yaxis=dict(zeroline=False),
        xaxis=dict(zeroline=False),
        autosize=False,
        width=1600,
        height=800,
    )

    fig = dict(data=data, layout=layout)
    py.iplot(fig, filename='styled-scatter')

    plt.show()

In [ ]:
# Plot 10 dimensions model in 2D
tsne_plot(model_10)

In [ ]:
# Re-train word2vec model with 300 dimensions
model_300 = Word2Vec(corpus, size=300, min_count=1)

In [ ]:
# Plot new vectors for the same words
fig = go.Figure(
    data=go.Heatmap(
        z=[model_300['control'], model_300['path'], model_300['anim'], model_300['deer'], model_300['run']],
        y=['control', 'path', 'anim', 'deer', 'run'],
    )
)
fig.show()

In [ ]:
#  Model understanding - We can obtain the word based on similarity
sim_words = model_300.wv.most_similar('deer')
sim_words

In [ ]:
sim_words = model_300.wv.most_similar('lefthand')
sim_words

In [ ]:
#  Model understanding - We can obtain the word based on similarity
sim_words = model_300.wv.most_similar('injur')
sim_words

In [ ]:
# Plot 300 dimensions model in 2D
tsne_plot(model_300)

## Doc2Vec Model

In [ ]:
# Preparing data for Doc2Vec Model
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Each document should have a list of words and a tag
list_id = list(case_text['CASENUMBER'])
list_def = list(case_text['text_string_fdist'])
tagged_data = [
    TaggedDocument(words=word_tokenize(term_def.lower()), tags=[list_id[i]])
    for i, term_def in enumerate(list_def)
]

# Let's take a look at the example
print(tagged_data[:2])

In [ ]:
# Instantiate the Doc2Vec Model
max_epochs = 50 # 250 iterations will take ~30 mins, ~8 sec per iteration
vec_size = 100
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha,
                min_alpha=0.00025,
                dm=1,
                min_count=2)
  
model.build_vocab(tagged_data)

In [ ]:
# Train the Doc2Vec Model
for epoch in range(max_epochs):
    if epoch % 10 == 0:
        print('iteration {0}'.format(epoch))

    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    
    model.alpha -= 0.0002
    model.min_alpha = model.alpha

model.save('d2v_do_v100_e50.model')

In [ ]:
# Check model results - word similarity
model.wv.similar_by_word('deer')

In [ ]:
model.wv.similar_by_word('fatal')

In [ ]:
model.wv.similar_by_word('injuri')

In [ ]:
# Check model results - doc similarity
similar_doc = model.docvecs.most_similar('1-32-2018-001-03')
print('1-32-2018-001-03', 'is most similar to:')
for i in range(len(similar_doc)):
    print(similar_doc[i][0])

In [ ]:
case_text[case_text['CASENUMBER'].isin(['1-32-2018-001-03', '1-21-2019-027-08', '1-32-2020-157-05', '1-11-2020-064-07', '1-32-2020-063-09', '1-32-2020-094-04'])]

In [ ]:
# Check the top 10 most similar documents for all documents
similarity_for_all =[]
for i in range(len(tagged_data)): 
  similarity_for_all.append(model.docvecs.most_similar(tagged_data[i][1][0]))

In [ ]:
similarity_for_all[0]

In [ ]:
# Reduce number of dimensions to 2D for visualization

nrows = 12495 # Use lower number to use a subset of documents to speed things up, the total number of documents is 12495
tsne = TSNE(perplexity=5, 
            n_components=2, 
            init='pca', 
            n_iter=1000, 
            random_state=123)
tsne_d2v = tsne.fit_transform(model.docvecs.vectors_docs[:nrows])


In [ ]:
# Add SUMMARY as x,y into a dataframe
tsne_d2v_df = pd.DataFrame(data=tsne_d2v, columns=['x', 'y'])  

# Add other columns
tsne_d2v_df['CATEGORY'] = case_text['CATEGORY'].values[:nrows]  
tsne_d2v_df['CRASHYEAR'] = case_text['CRASHYEAR'].values[:nrows]
tsne_d2v_df['CRASHMONTH'] = case_text['CRASHMONTH'].values[:nrows]
tsne_d2v_df['CINJURED'] = case_text['CINJURED'].values[:nrows]
tsne_d2v_df['CRASHTIME'] = case_text['CRASHTIME'].values[:nrows]
tsne_d2v_df['DAYOFWEEK'] = case_text['DAYOFWEEK'].values[:nrows]
tsne_d2v_df['CINJSEV'] = case_text['CINJSEV'].values[:nrows]
tsne_d2v_df['SUMMARY'] = case_text['SUMMARY'].values[:nrows]

# Add INJURED column
tsne_d2v_df["INJURED"] = np.where(tsne_d2v_df["CINJURED"] == 0, 0, 1)

# Add WINTER column
seasons = {
    1: "1",
    2: "1",
    3: "0",
    4: "0",
    5: "0",
    6: "0",
    7: "0",
    8: "0",
    9: "0",
    10: "0",
    11: "0",
    12: "1",
}
tsne_d2v_df["WINTER"] = tsne_d2v_df["CRASHMONTH"].apply(lambda x: seasons[x])

# Add NIGHT column
tsne_d2v_df["NIGHT"] = tsne_d2v_df["CRASHTIME"].apply(
    lambda x: "1" if "00:00" <= x <= "06:00" or "20:00" <= x <= "23:00" else "0"
)

# Add weekend column
tsne_d2v_df["WEEKEND"] = tsne_d2v_df["DAYOFWEEK"].apply(
    lambda x: "1" if 6 <= x <= 7 else "0"
)

In [ ]:
# Check top 3 rows
tsne_d2v_df.head(3)

In [ ]:
hist = tsne_d2v_df.hist(column='CRASHYEAR')

In [ ]:
from bokeh.models import ColumnDataSource, LabelSet, HoverTool
from bokeh.plotting import figure, show
from bokeh.io import show, output_notebook
from bokeh.palettes import Magma, Inferno, Plasma, Viridis, OrRd3, Turbo # You can try different pallets
from bokeh.models import LinearColorMapper

output_notebook()

exp_cmap = LinearColorMapper(palette='Turbo256', 
                             low = min(tsne_d2v_df.CRASHYEAR.astype(int)), 
                             high = max(tsne_d2v_df.CRASHYEAR.astype(int)))

source = ColumnDataSource(tsne_d2v_df)

TOOLS='hover,crosshair,pan,box_zoom,undo,reset'
TOOLTIPS = [
    ('SUMMARY', '@SUMMARY'),
]

p = figure(plot_height =600, plot_width = 1200, tools=TOOLS, tooltips=TOOLTIPS)
p.scatter(x='x', y='y', source=source, size=8, fill_color={'field':'CRASHYEAR', 'transform':exp_cmap}, alpha=0.5, legend_field='CRASHYEAR')

show(p)

In [ ]:
hist = tsne_d2v_df.hist(column='CRASHMONTH', bins=12)

In [ ]:
output_notebook()

exp_cmap = LinearColorMapper(palette='Turbo256', 
                             low = min(tsne_d2v_df.CRASHMONTH.astype(int)), 
                             high = max(tsne_d2v_df.CRASHMONTH.astype(int)))

source = ColumnDataSource(tsne_d2v_df)

p = figure(plot_height = 600, plot_width = 1200, tools=TOOLS, tooltips=TOOLTIPS)
p.scatter(x='x', y='y', source=source, size=8, fill_color={'field':'CRASHMONTH', 'transform':exp_cmap}, alpha=0.5, legend_field='CRASHMONTH')

show(p)